In [1]:
#Import neccessary libraries
import numpy as np
import pandas as pd
import plotly.express as px
import seaborn as sn
import matplotlib.pyplot as plt

In [2]:
#Import the pokemon data
pokemon = pd.read_csv("pokemon.csv")
pokemon.head()

,#,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,80,82,83,100,100,80,1,False
3,4,Mega Venusaur,Grass,Poison,80,100,123,122,120,80,1,False
4,5,Charmander,Fire,NaN,39,52,43,60,50,65,1,False


In [3]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
pokemon.Legendary = le.fit_transform(pokemon.Legendary)
pokemon.head(10)

,#,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,45,49,49,65,65,45,1,0
1,2,Ivysaur,Grass,Poison,60,62,63,80,80,60,1,0
2,3,Venusaur,Grass,Poison,80,82,83,100,100,80,1,0
3,4,Mega Venusaur,Grass,Poison,80,100,123,122,120,80,1,0
4,5,Charmander,Fire,NaN,39,52,43,60,50,65,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
795,796,Diancie,Rock,Fairy,50,100,150,100,150,50,6,1
796,797,Mega Diancie,Rock,Fairy,50,160,110,160,110,110,6,1
797,798,Hoopa Confined,Psychic,Ghost,80,110,60,150,130,70,6,1
798,799,Hoopa Unbound,Psychic,Dark,80,160,60,170,130,80,6,1


In [4]:
#Import combat data for the pokemon
combat = pd.read_csv("combats.csv")
combat.head()

,First_pokemon,Second_pokemon,Winner
0,266,298,298
1,702,701,701
2,191,668,668
3,237,683,683
4,151,231,151


In [5]:
# Import type effectiveness chart
# This chart is used to check the a Pokemon's type against its opponent
type_chart = pd.read_csv('typecharteffectiveness.csv')
type_chart = type_chart.set_index('Attacking')
type_chart.head()

,Normal,Fire,Water,Electric,Grass,Ice,Fighting,Poison,Ground,Flying,Psychic,Bug,Rock,Ghost,Dragon,Dark,Steel,Fairy
Attacking,,,,,,,,,,,,,,,,,,
Normal,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.5,0.0,1.0,1.0,0.5,1.0
Fire,1,0.5,0.5,1.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,2.0,0.5,1.0,0.5,1.0,2.0,1.0
Water,1,2.0,0.5,1.0,0.5,1.0,1.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0,0.5,1.0,1.0,1.0
Electric,1,1.0,2.0,0.5,0.5,1.0,1.0,1.0,0.0,2.0,1.0,1.0,1.0,1.0,0.5,1.0,1.0,1.0
Grass,1,0.5,2.0,1.0,0.5,1.0,1.0,0.5,2.0,0.5,1.0,0.5,2.0,1.0,0.5,1.0,0.5,1.0


In [15]:
extra_combat_df = combat
extra_combat_df.head(10)

,First_pokemon,Second_pokemon,Winner
0,266,298,298
1,702,701,701
2,191,668,668
3,237,683,683
4,151,231,151
5,657,752,657
6,192,134,134
7,73,545,545
8,220,763,763
9,302,31,31


In [7]:
combat.shape

(50000, 3)

In [8]:
# This generator is used to help manage the iteration of a dataframe
def df_generator(df):
    for _, row in df.iterrows():
        yield row

In [9]:
simple_gen = df_generator(combat)
poke_match = next(simple_gen)
# poke = pokemon[pokemon['#'] == first['First_pokemon']]
poke1 = pokemon.loc[poke_match['First_pokemon']-1]
poke2 = pokemon.loc[poke_match['Second_pokemon']-1]
print(poke1['Attack'])
print(poke2['Attack'])
effectiveness = type_chart.loc[poke1['Type 1']][poke2['Type 1']]
hp_diff = poke1['HP'] - poke2['HP']
attack_diff = poke1['Attack'] - poke2['Attack']
attack_diff

64
70


-6

In [12]:
# This is used to make a list of Series with the difference in stats between the pokemon in the combats.csv
# first_poke(features) - second_poke(features) = battle_stats
my_list = []
for index, row in df_generator(combat):
    dict_battle_stats = {}
    first_poke = pokemon.loc[row['First_pokemon']-1]
    second_poke = pokemon.loc[row['Second_pokemon']-1]
    f_type = first_poke['Type 1']
    s_type = second_poke['Type 1']
    dict_battle_stats['Effectiveness'] = type_chart[f_type][s_type]
    dict_battle_stats['HP_diff'] = first_poke['HP'] - second_poke['HP']
    dict_battle_stats['Attack_diff'] = first_poke['Attack'] - second_poke['Attack']
    dict_battle_stats['Defense_diff'] = first_poke['Defense'] - second_poke['Defense']
    dict_battle_stats['Sp.Atk_diff'] = first_poke['Sp. Atk'] - second_poke['Sp. Atk']
    dict_battle_stats['Sp.Def_diff'] = first_poke['Sp. Def'] - second_poke['Sp. Def']
    dict_battle_stats['Speed_diff'] = first_poke['Speed'] - second_poke['Speed']
    dict_battle_stats['F_Legendary'] = first_poke['Legendary']
    dict_battle_stats['S_Legendary'] = second_poke['Legendary']
    dict_battle_stats['FirstIsWinner'] = 1 if row['First_pokemon'] == row['Winner'] else 0
    new_series = pd.Series(dict_battle_stats)
    my_list.append(new_series)

In [14]:
# This is used to make a Dataframe from the recently created list
extra_feats_df = pd.DataFrame(my_list, columns=['Effectiveness', 'HP_diff', 'Attack_diff', 'Defense_diff', 'Sp.Atk_diff', 'Sp.Def_diff', 'Speed_diff', 'F_Legendary', 'S_Legendary', 'FirstIsWinner'])
extra_feats_df.head(10)

,Effectiveness,HP_diff,Attack_diff,Defense_diff,Sp.Atk_diff,Sp.Def_diff,Speed_diff,F_Legendary,S_Legendary,FirstIsWinner
0,2.0,-20.0,-6.0,10.0,-15.0,10.0,-19.0,0.0,0.0,0.0
1,1.0,0.0,-39.0,-18.0,18.0,39.0,0.0,1.0,1.0,0.0
2,1.0,-20.0,-35.0,10.0,-45.0,10.0,0.0,0.0,0.0,0.0
3,1.0,-37.0,-80.0,-50.0,10.0,-50.0,-28.0,0.0,0.0,0.0
4,1.0,50.0,50.0,-105.0,105.0,-160.0,50.0,0.0,0.0,1.0
5,1.0,-10.0,-3.0,-100.0,7.0,-100.0,5.0,0.0,0.0,1.0
6,1.0,-25.0,0.0,10.0,-45.0,-50.0,-25.0,0.0,0.0,0.0
7,1.0,-80.0,-20.0,-70.0,-65.0,-85.0,-55.0,0.0,1.0,0.0
8,1.0,0.0,12.0,28.0,-23.0,-28.0,-29.0,0.0,0.0,0.0
9,2.0,5.0,-25.0,-10.0,5.0,-20.0,-5.0,0.0,0.0,0.0


In [ ]:
# change types to correct types
extra_feats_df['HP_diff'] = extra_feats_df['HP_diff'].astype(int)
extra_feats_df['Attack_diff'] = extra_feats_df['Attack_diff'].astype(int)
extra_feats_df['Defense_diff'] = extra_feats_df['Defense_diff'].astype(int)
extra_feats_df['Sp.Atk_diff'] = extra_feats_df['Sp.Atk_diff'].astype(int)
extra_feats_df['Sp.Def_diff'] = extra_feats_df['Sp.Def_diff'].astype(int)
extra_feats_df['Speed_diff'] = extra_feats_df['Speed_diff'].astype(int)
extra_feats_df['F_Legendary'] = extra_feats_df['F_Legendary'].astype(int)
extra_feats_df['S_Legendary'] = extra_feats_df['S_Legendary'].astype(int)
extra_feats_df['First Won'] = extra_feats_df['First Won'].astype(int)
extra_feats_df.head(10)

In [ ]:
extra_feats_df.shape

In [ ]:
extra_combat_df = pd.concat([extra_combat_df, extra_feats_df], axis='columns')
extra_combat_df

In [ ]:
# uncomment if you need to test

# first_poke = pokemon[pokemon['#'] == 266]
# second_poke = pokemon[pokemon['#'] == 298]
# fake_df2 = pd.DataFrame(columns=['Effectiveness', 'HP_diff', 'Attack_diff', 'Defense_diff', 'Sp.Atk_diff', 'Sp.Def_diff', 'Speed_diff', 'F_Legendary', 'S_Legendary'])
# li_battle_stats2 = {}
# f_type = first_poke['Type 1'].to_string(index=False)
# s_type = second_poke['Type 1'].to_string(index=False)
# li_battle_stats2['Effectiveness'] = float(type_chart[type_chart['Attacking'] == f_type][s_type].to_string(index=False))
# li_battle_stats2['HP_diff'] = int(first_poke['HP'].to_string(index=False)) - int(second_poke['HP'].to_string(index=False))
# li_battle_stats2['Attack_diff'] = int(first_poke['Attack'].to_string(index=False)) - int(second_poke['Attack'].to_string(index=False))
# li_battle_stats2['Defense_diff'] = int(first_poke['Defense'].to_string(index=False)) - int(second_poke['Defense'].to_string(index=False))
# li_battle_stats2['Sp.Atk_diff'] = int(first_poke['Sp. Atk'].to_string(index=False)) - int(second_poke['Sp. Atk'].to_string(index=False))
# li_battle_stats2['Sp.Def_diff'] = int(first_poke['Sp. Def'].to_string(index=False)) - int(second_poke['Sp. Def'].to_string(index=False))
# li_battle_stats2['Speed_diff'] = int(first_poke['Speed'].to_string(index=False)) - int(second_poke['Speed'].to_string(index=False))
# li_battle_stats2['F_Legendary'] = bool(first_poke['Legendary'].to_string(index=False))
# li_battle_stats2['S_Legendary'] = bool(second_poke['Legendary'].to_string(index=False))
# new_series = pd.Series(li_battle_stats2)
# fake_df2 = fake_df2.append(new_series, ignore_index=True)
# fake_df2

In [ ]:
#Using describe function to show mean, min, max, etc...
pokemon.describe()

Cleaning up missing and duplicate data

In [ ]:
pokemon.isnull().sum()

In [ ]:
print(pokemon[pokemon['Name'].isnull()])

In [ ]:
pokemon.at[61, 'Name']

In [ ]:
#62 should be the evolution of 61. 61 is Mankey, and Primeape is the evolved version of Mankey
#Knowing this, set the name of 62 to Primeape.
pokemon.at[62, 'Name'] = "Primeape"

In [ ]:
#Keep only the first of duplicate records for the pokemon and combat files
pokemon.duplicated(keep='first')

In [ ]:
#Check for any null data in the combat file
combat.isnull().sum()

In [ ]:
combat.duplicated(keep='first')

Finding the win percentage, and merging win percentage with Pokemon data

In [ ]:
combat.head()

In [ ]:
#Count the number of wins by each Pokemon
wins = combat.groupby('Winner').count()
#Drop the Second_pokemon column
wins = wins.drop('Second_pokemon', axis = 1)
#Rename First_pokemon column to Number_of_wins
wins = wins.rename(columns={'First_pokemon': 'Number_of_wins'})
wins

In [ ]:
#Get the count of battles for the first and second pokemon in the combat csv
firstPokemonCount = combat.groupby('Second_pokemon').count()
secondPokemonCount = combat.groupby('First_pokemon').count()
#Add the resulting counts together to get the total battles, add the results to wins
wins['Num_Fights'] = firstPokemonCount.Winner + secondPokemonCount.Winner
#Get the win % and add the results to wins table
wins['Win_%'] = wins.Number_of_wins/wins['Num_Fights']
wins

In [ ]:
#Merge the wins table with the original Pokemon data
result = pd.merge(pokemon, wins, left_on='#', right_index = True, how = 'left')
result.columns = result.columns.str.strip()
result

In [ ]:
#Create new table showing Pokemon stats and their correlated win percentage
feats = ['HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed', 'Win_%']
corr = result[feats]
corr

In [ ]:
#Create a stats matrix, and visualize the data
df_corr = corr.corr()
df_corr

In [ ]:
#Show heat map of correlation matrix for stats
heatmap = go.Figure()
heatmap.add_trace(
    go.Heatmap(
        x = df_corr.columns,
        y = df_corr.index,
        z = np.array(df_corr)
    )
)

In [ ]:
sn.heatmap(df_corr, annot=True)

In [ ]:
#check for any missing data
fitting_df = result[feats]
fitting_df.isna().sum()

In [ ]:
# These are the Pokemon with missing combat values
missing = result.drop('Type 2', axis='columns')
missing[missing.isna().any(axis='columns')]

In [ ]:
#remove the ones with the missing data
fitting_df.dropna(inplace=True)
fitting_df.isna().sum()

In [ ]:
fitting_df.shape

In [ ]:
X = fitting_df.drop(['Win_%'], axis='columns')
X.head()

In [ ]:
y = fitting_df['Win_%']
y.head()

In [ ]:
# Splitting the dataframe into a Train set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=3)

In [ ]:
len(X_train)

In [ ]:
len(X_test)

In [ ]:
from sklearn.linear_model import LinearRegression
model_lr = LinearRegression()
model_lr.fit(X_train, y_train)

In [ ]:
model_lr.score(X_test, y_test)

In [ ]:
from sklearn.metrics import mean_absolute_error

In [ ]:
y_predicted = model_lr.predict(X_test)
mae = mean_absolute_error(y_test, y_predicted)
mae

In [ ]:
extra_feats_df.head()

In [ ]:
X = extra_feats_df.drop(['First Won'], axis='columns')
X.head()

In [ ]:
y = extra_feats_df['First Won']
y

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=4)

In [ ]:
len(X_train)

In [ ]:
len(X_test)

In [ ]:
from sklearn.linear_model import LogisticRegression
model_logreg = LogisticRegression()

In [ ]:
model_logreg.fit(X_train, y_train)

In [ ]:
model_logreg.score(X_test, y_test)

In [ ]:
Solgaleo = {
    "Type 1": 'Psychic',
    'HP': 137,
    'Attack': 137,
    'Defense': 107,
    'Sp. Atk': 113,
    'Sp. Def': 89,
    'Speed': 97,
    'Legendary': 1,
}
Corvisquire = {
    "Type 1": 'Flying',
    'HP': 68,
    'Attack':67,
    'Defense': 55,
    'Sp. Atk': 43,
    'Sp. Def': 55,
    'Speed': 77,
    'Legendary': 0,
}

In [ ]:
def pokemon_vs_dict(poke1, poke2):
    stats = {}
    stats['Effectiveness'] = float(type_chart[type_chart['Attacking'] == poke1['Type 1']][poke2['Type 1']].to_string(index=False))
    stats['F_Legendary'] = poke1['Legendary']
    stats['S_Legendary'] = poke2['Legendary']
    for key, val in poke1.items():
        if key != 'Type 1' and key != 'Legendary':
#             print(f'{key} {val}')
            stats[key] = int(val - poke2[key])
    return stats

In [ ]:
def predict_pokemon_win(battle_stats):
    stats = [battle_stats['Effectiveness'], battle_stats['HP'], battle_stats['Attack'],battle_stats['Defense'],
             battle_stats['Sp. Atk'], battle_stats['Sp. Def'], battle_stats['Speed'], battle_stats['F_Legendary'],
             battle_stats['S_Legendary']]
    return model_logreg.predict_proba([stats])

In [ ]:
pokemon[pokemon['#'] == 56]

In [ ]:
battle_stats1 = pokemon_vs_dict(Solgaleo, Corvisquire)
predict_pokemon_win(battle_stats1)

In [ ]:
Meowscarada = {
   'Type 1': 'Grass',
    'HP': 76,
    'Attack': 110,
    'Defense': 70,
    'Sp. Atk': 81,
    'Sp. Def': 70,
    'Speed': 123,
    'Legendary': 0, 
}
Greninja = {
   'Type 1': 'Water',
    'HP': 72,
    'Attack': 95,
    'Defense': 67,
    'Sp. Atk': 103,
    'Sp. Def': 71,
    'Speed': 122,
    'Legendary': 0, 
}

In [ ]:
battle_stats2 = pokemon_vs_dict(Meowscarada, Greninja)
predict_pokemon_win(battle_stats2)

In [ ]:
Mew = {
   'Type 1': 'Psychic',
    'HP': 100,
    'Attack': 100,
    'Defense': 100,
    'Sp. Atk': 100,
    'Sp. Def': 100,
    'Speed': 100,
    'Legendary': 1, 
}
Mewtwo = {
   'Type 1': 'Psychic',
    'HP': 106,
    'Attack': 110,
    'Defense': 90,
    'Sp. Atk': 154,
    'Sp. Def': 90,
    'Speed': 130,
    'Legendary': 1, 
}

In [ ]:
battle_stats3 = pokemon_vs_dict(Mew, Mewtwo)
predict_pokemon_win(battle_stats3)

In [ ]:
Pikachu = {
   'Type 1': 'Electric',
    'HP': 35,
    'Attack': 55,
    'Defense': 40,
    'Sp. Atk': 50,
    'Sp. Def': 50,
    'Speed': 90,
    'Legendary': 0, 
}
Eevee = {
   'Type 1': 'Normal',
    'HP': 55,
    'Attack': 55,
    'Defense': 50,
    'Sp. Atk': 45,
    'Sp. Def': 65,
    'Speed': 55,
    'Legendary': 0, 
}

In [ ]:
battle_stats4 = pokemon_vs_dict(Pikachu, Eevee)
predict_pokemon_win(battle_stats4)